# Integração entre Dados Vetoriais e Matriciais

Se você já fez análises combinando shapefiles com imagens de satélite ou MDTs, aqui vamos aprender a automatizar esse processo! É como ter um assistente que combina diferentes tipos de dados para criar análises complexas.

## Objetivos
- Combinar dados vetoriais e matriciais (como sobrepor camadas)
- Extrair estatísticas de rasters por polígono (como zonal statistics)
- Converter entre formatos (como rasterização e vetorização)
- Realizar análises espaciais integradas (como modelagem ambiental)

## 1. Configuração do Ambiente

Vamos preparar nosso laboratório de análise integrada:

In [ ]:
# Nossa estação de trabalho integrada:
import geopandas as gpd          # Para dados vetoriais
import rasterio                  # Para dados matriciais
from rasterio.mask import mask   # Para recortar rasters
import numpy as np               # Para cálculos
import matplotlib.pyplot as plt   # Para visualização
from rasterstats import zonal_stats  # Para estatísticas por zona
import pandas as pd              # Para tabelas
from shapely.geometry import box  # Para criar retângulos

plt.rcParams['figure.figsize'] = (12, 8)

## 2. Extração de Valores de Raster por Polígono

Como calcular estatísticas de uma imagem ou MDT para cada área de interesse:

In [ ]:
# Carregando dados
# Como abrir um shapefile e um raster
estados = gpd.read_file('../data/raw/BR_UF_2022.shp')

# Função para extrair estatísticas
# Como fazer Zonal Statistics
def extrair_estatisticas_raster(raster_path, poligonos):
    """Calcula estatísticas do raster para cada polígono"""
    stats = zonal_stats(poligonos,
                       raster_path,
                       stats=['min', 'max', 'mean', 'std'])
    return pd.DataFrame(stats)

# Extraindo estatísticas
# Como calcular valores médios por estado
stats_df = extrair_estatisticas_raster('../data/processed/exemplo.tif', estados)

# Combinando com os dados dos estados
# Como juntar resultados na tabela de atributos
estados_stats = estados.copy()
estados_stats['valor_medio'] = stats_df['mean']
estados_stats['valor_std'] = stats_df['std']

# Visualização
# Como fazer mapas dos resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

estados_stats.plot(column='valor_medio',
                  cmap='viridis',
                  legend=True,
                  ax=ax1)
ax1.set_title('Valor Médio por Estado')

estados_stats.plot(column='valor_std',
                  cmap='Reds',
                  legend=True,
                  ax=ax2)
ax2.set_title('Variabilidade por Estado')

plt.tight_layout()
plt.show()

## 3. Rasterização de Dados Vetoriais

Como transformar shapefiles em rasters:

In [ ]:
from rasterio.features import rasterize

def rasterizar_vetor(gdf, valor_coluna, resolucao=0.1):
    """Converte dados vetoriais para raster (como Feature to Raster)"""
    # Definindo a transformação
    # Como configurar o tamanho do pixel
    bounds = gdf.total_bounds
    width = int((bounds[2] - bounds[0]) / resolucao)
    height = int((bounds[3] - bounds[1]) / resolucao)
    transform = rasterio.transform.from_bounds(*bounds, width, height)
    
    # Preparando as geometrias e valores
    # Como associar valores aos polígonos
    shapes = ((geom, value) 
              for geom, value in zip(gdf.geometry, gdf[valor_coluna]))
    
    # Rasterizando
    # Como criar a matriz de pixels
    raster = rasterize(shapes=shapes,
                       out_shape=(height, width),
                       transform=transform,
                       fill=0,
                       all_touched=True,
                       dtype=float)
    
    return raster, transform

# Rasterizando a densidade populacional
# Como criar um raster de densidade
raster_densidade, transform = rasterizar_vetor(estados_stats, 'densidade')

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Dados vetoriais originais
estados_stats.plot(column='densidade',
                  cmap='viridis',
                  legend=True,
                  ax=ax1)
ax1.set_title('Densidade - Vetorial')

# Dados rasterizados
im = ax2.imshow(raster_densidade,
                cmap='viridis',
                extent=[estados_stats.total_bounds[0],
                       estados_stats.total_bounds[2],
                       estados_stats.total_bounds[1],
                       estados_stats.total_bounds[3]])
ax2.set_title('Densidade - Raster')
plt.colorbar(im, ax=ax2)

plt.tight_layout()
plt.show()

## 4. Análise Integrada

Como combinar diferentes tipos de dados para análises complexas:

In [ ]:
# Função para criar índice de adequabilidade
# Como fazer análise multicritério
def calcular_adequabilidade(raster_valor, raster_densidade, pesos=(0.6, 0.4)):
    """Combina diferentes critérios com pesos"""
    # Normalizando os rasters
    # Como padronizar diferentes medidas
    valor_norm = (raster_valor - raster_valor.min()) / (raster_valor.max() - raster_valor.min())
    densidade_norm = (raster_densidade - raster_densidade.min()) / (raster_densidade.max() - raster_densidade.min())
    
    # Calculando o índice ponderado
    # Como combinar critérios com pesos
    adequabilidade = (valor_norm * pesos[0] + densidade_norm * pesos[1])
    
    return adequabilidade

# Calculando adequabilidade
with rasterio.open('../data/processed/exemplo.tif') as src:
    raster_valor = src.read(1)

adequabilidade = calcular_adequabilidade(raster_valor, raster_densidade)

# Extraindo estatísticas por estado
estados_stats['adequabilidade_media'] = extrair_estatisticas_raster(
    '../data/processed/exemplo.tif', estados_stats)['mean']

# Visualização
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))

# Critério 1
im1 = ax1.imshow(raster_valor, cmap='YlOrRd')
ax1.set_title('Critério 1 - Valores')
plt.colorbar(im1, ax=ax1)

# Critério 2
im2 = ax2.imshow(raster_densidade, cmap='viridis')
ax2.set_title('Critério 2 - Densidade')
plt.colorbar(im2, ax=ax2)

# Resultado da combinação
im3 = ax3.imshow(adequabilidade, cmap='RdYlGn')
ax3.set_title('Índice de Adequabilidade')
plt.colorbar(im3, ax=ax3)

# Média por estado
estados_stats.plot(column='adequabilidade_media',
                  cmap='RdYlGn',
                  legend=True,
                  ax=ax4)
ax4.set_title('Adequabilidade Média por Estado')

plt.tight_layout()
plt.show()

## 5. Análise de Mudanças

Como detectar e quantificar mudanças ao longo do tempo:

In [ ]:
# Simulando mudança temporal
# Como detectar mudanças entre datas
def simular_mudanca(raster, taxa_mudanca=0.1):
    """Simula mudança temporal em um raster"""
    return raster * (1 + np.random.normal(0, taxa_mudanca, raster.shape))

# Simulando dois períodos
# Como ter duas datas para comparar
raster_t1 = raster_valor
raster_t2 = simular_mudanca(raster_t1)

# Calculando mudança
# Como fazer diferença entre datas
mudanca = raster_t2 - raster_t1

# Extraindo estatísticas de mudança
# Como calcular mudança média por estado
estados_stats['mudanca_media'] = (mudanca.mean())

# Visualização
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))

# Data 1
im1 = ax1.imshow(raster_t1, cmap='viridis')
ax1.set_title('Período 1')
plt.colorbar(im1, ax=ax1)

# Data 2
im2 = ax2.imshow(raster_t2, cmap='viridis')
ax2.set_title('Período 2')
plt.colorbar(im2, ax=ax2)

# Mapa de mudanças
im3 = ax3.imshow(mudanca, cmap='RdBu')
ax3.set_title('Mudança')
plt.colorbar(im3, ax=ax3)

# Mudança por estado
estados_stats.plot(column='mudanca_media',
                  cmap='RdBu',
                  legend=True,
                  ax=ax4)
ax4.set_title('Mudança Média por Estado')

plt.tight_layout()
plt.show()

## 6. Exercícios Práticos

Vamos praticar como um analista GIS avançado:

1. Análise Integrada:
   - Combine MDT com uso do solo
   - Calcule estatísticas por município
   - Crie um índice de risco
   - Como análise ambiental

2. Análise Temporal:
   - Use imagens de diferentes datas
   - Calcule índices de vegetação
   - Analise mudanças por região
   - Como monitoramento ambiental

3. Modelagem Espacial:
   - Desenvolva modelo de adequabilidade
   - Combine diferentes variáveis
   - Valide os resultados
   - Como planejamento territorial